In [23]:
from http import client
from urllib import response
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from img2vec_pytorch import Img2Vec
from sentence_transformers import SentenceTransformer
import plotly.express as px
import plotly.graph_objects as go
import urllib.request
import requests
import pandas as pd
import json
import pprint
import seaborn as sns
import matplotlib.pyplot as plt
import json
import numpy as np

In [ ]:
description_list = []
contract_addresses = []
name_list = []

for j in range(30):
  with open(f'/home/dustin/Documents/Ducia/NFT_image_recommendation/Blockdaemon_NFT_Contracts/{j+1}.json') as json_file:
      json_data = json.load(json_file)['data']

  address_list = []
  for i in json_data:
      if ("contracts" in i.keys() and len(i["contracts"]) > 0 and "address" in i["contracts"][0].keys()):
          address = i["contracts"][0]["address"]
          address_list.append(address)

  print("new file yay :)")

  counter = 0
  for address in address_list:
      counter += 1
      #print(counter)
      # evtl muessen wir checken ob alles erc721 sind und tokenID 1 immer existiert
      URL = f"https://eth-mainnet.alchemyapi.io/v2/demo/getNFTMetadata?contractAddress={address}&tokenId=1&tokenType=erc721"
      NFT = requests.get(URL)
      try:
          NFTjson = NFT.json()
      except:
          continue
      if (type(NFTjson) is dict and "metadata" in NFTjson.keys() and type(NFTjson["metadata"]) is dict and "description" in NFTjson["metadata"].keys()):
          print('This is what we want')
          external_url = NFTjson["metadata"]["description"]
          description_list.append(external_url)
          contract_addresses.append(address)
          name = "Not Available"
          if (type(NFTjson["metadata"]) is dict and "title" in NFTjson["metadata"].keys()):
            name = NFTjson["metadata"]["title"]
          name_list.append(name)


In [24]:
#np.savez('/home/dustin/Documents/Ducia/NFT_image_recommendation/NFT_attributes.npz', description_list=description_list, contract_addresses=contract_addresses, name_list=name_list)
NFT_attributes = np.load('/home/dustin/Documents/Ducia/NFT_image_recommendation/NFT_attributes.npz')
d = dict(zip(("description_list","contract_addresses","name_list"), (NFT_attributes[k] for k in NFT_attributes)))
description_list = d['description_list']
contract_addresses = d['contract_addresses']
name_list = d['name_list']

In [4]:
#print('Loading Model')
model = SentenceTransformer('bert-base-nli-mean-tokens')

#description embeddings
print(f'Calculating embeddings for {len(description_list)} descriptions')
desc_embeddings = model.encode(description_list, normalize_embeddings=True, show_progress_bar=True, batch_size = 64)

#desc_embeddings

Calculating embeddings for 11083 descriptions


Batches: 100%|██████████| 174/174 [13:56<00:00,  4.81s/it]


In [25]:
#np.savez('/home/dustin/Documents/Ducia/NFT_image_recommendation/desc_emb.npz', desc_embeddings=desc_embeddings)
desc_emb = np.load('/home/dustin/Documents/Ducia/NFT_image_recommendation/desc_emb.npz')
desc_embeddings = dict(zip(("desc_embeddings"), (desc_emb[k] for k in desc_emb)))['d']
print(desc_embeddings.shape)

(11083, 768)


In [26]:
#Test
desc_embeddings = desc_embeddings[:1000]
description_list = description_list[:1000]
contract_addresses = contract_addresses[:1000]
name_list = name_list[:1000]

In [27]:
nft_tsne = TSNE(n_components=3, n_jobs = -1).fit_transform(desc_embeddings)
nft_tsne_df = pd.DataFrame(nft_tsne)
nft_tsne_df = (nft_tsne_df-nft_tsne_df.mean())/nft_tsne_df.std()
nft_tsne_df

/home/dustin/anaconda3/envs/web3/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/home/dustin/anaconda3/envs/web3/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


,0,1,2
0,0.028098,-1.437364,-0.842759
1,-1.429028,-0.785536,1.110220
2,-2.184209,-0.135999,0.971122
3,0.050036,-0.089496,-1.141788
4,0.335026,-1.151593,1.005567
...,...,...,...
995,1.427649,1.173030,0.344386
996,1.390254,0.934677,0.446467
997,1.211344,1.104285,0.433882
998,1.218565,0.861361,0.310416


In [31]:
truncated_descriptions = [desc[0:50] + '...' if len(desc) > 50 else desc for desc in description_list]
final_df = pd.concat([nft_tsne_df, pd.DataFrame(contract_addresses), pd.DataFrame(name_list), pd.DataFrame(truncated_descriptions)], axis = 1)
final_df.columns = ['X', 'Y', 'Z', 'NFT_Address', 'Name', 'D']
final_df

,X,Y,Z,NFT_Address,Name,D
0,0.028098,-1.437364,-0.842759,0x14232931f4151a7a962c1929F670378159d87fb7,Not Available,Algorithmically generated massive amount of da...
1,-1.429028,-0.785536,1.110220,0xC4eaEAfa1E796a2a0E92aDA206499E0c9e9a3B3a,Not Available,
2,-2.184209,-0.135999,0.971122,0x8E34450a3C674d47B6494f346Ac3D67f8c8dfc15,Not Available,Memory.\n\nThe gods you knew before are dying ...
3,0.050036,-0.089496,-1.141788,0x7a414ECfcD7DAd240B1E6dD694a60F956425967e,Not Available,“Death comes for us all. Even for kings he com...
4,0.335026,-1.151593,1.005567,0xC52877e8b234307f2bA61AC0b0Eb64e5DBA0719f,Not Available,"36.0504738, -121.5919463\n\nGamboa Point\nPaci..."
...,...,...,...,...,...,...
995,1.427649,1.173030,0.344386,0x95B82f2159Ad60f6a4D8Feb4C101a768Ed6643cF,Not Available,[Angels and Devils](https://angelsdevilsnft.in...
996,1.390254,0.934677,0.446467,0x9E60eF2dA8e374cbc0eD14A839a3F0D8d6dfC651,Not Available,[Angels and Devils](https://angelsdevilsnft.in...
997,1.211344,1.104285,0.433882,0x08A3EC8348065B8d0ADc0394165c960ddC6cd2F5,Not Available,[Angels and Devils](https://angelsdevilsnft.in...
998,1.218565,0.861361,0.310416,0x42Cbe37F9955AE3191cDb3289125d120d5844E08,Not Available,[Angels and Devils](https://angelsdevilsnft.in...


In [ ]:
fig = px.scatter_3d(
    final_df, x= 'X', y= 'Y', z= 'Z',
    size=[1]*len(final_df),
    color = 'NFT_Address', 
    labels= {'influence_rating': 'Legend'},
    hover_data = 'D',
)
fig.show()

In [ ]:
fig.update_layout(xaxis=dict(showspikes=False) , 
        width=2000,
        height=1000,
        margin=dict(
            l=2,
            r=2,
            b=2,
            t=2,
            pad=4
        ), )

fig.update_traces(marker_size = 3 )
fig.update_coloraxes(showscale=False)
fig.update_scenes(xaxis_visible=False, yaxis_visible=False,zaxis_visible=False )

fig.write_html("attributes+description_pca.html")
fig.show()

In [ ]:
# Testing Random Neighborhood
index = 0
sample = desc_embeddings[index]
for di in desc_embeddings:
    distance = (sample - di).square() #np.linalg.norm(desc_embeddings[sample] - di)
